In [1]:
import h5py as h5
import numpy as np
import tensorflow as tf

2023-06-16 09:42:12.256011: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-16 09:42:13.027548: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/albert/miniconda3/envs/Baikal2/lib/:/home/albert/miniconda3/envs/Baikal2/lib/:/home/albert/miniconda3/envs/Baikal2/lib/:/home/albert/miniconda3/envs/Baikal2/lib/:/home/albert/miniconda3/envs/Baikal2/lib/:/home/albert/miniconda3/envs/Baikal2/lib/:/home/albert/miniconda3/envs/Baikal2/lib/:/home/albert/miniconda3/envs/Baikal2/lib/:/home/albert/miniconda3/envs/Baikal2/lib/
2023-06-16 09:42:13.02

In [2]:
# mult noise
class gauss_mult_noise:
    def __init__(self, Q_mean_noise, n_fraction):
        self.Q_mean_noise = Q_mean_noise
        self.n_fraction = n_fraction
    def make_noise(self, Qs, mask):
        noises = np.random.normal( scale=self.n_fraction, size=Qs.shape )*mask
        Qs = Qs + (Qs+self.Q_mean_noise)*noises
        return Qs

# addative noise
def add_gauss(data_in, g_add_stds):
    data = data_in[:,:,:-1]
    mask = data_in[:,:,-1:]
    g_add_stds = np.broadcast_to(g_add_stds, data.shape)
    noise = np.random.normal( scale=g_add_stds, size=data.shape )*mask
    data += noise
    sort_idxs = np.argsort(data[:,:,1:2], axis=1)
    data = np.take_along_axis( data, sort_idxs, axis=1)
    data = np.concatenate((data,mask), axis=-1)
    return data

def add_gauss(data_in, g_add_stds):
    data = data_in[:,:,:-1]
    mask = data_in[:,:,-1:]
    g_add_stds = np.broadcast_to(g_add_stds, data.shape)
    noise = np.random.normal( scale=g_add_stds, size=data.shape )*mask
    data += noise
    sort_idxs = np.argsort(data[:,:,1:2], axis=1)
    data = np.take_along_axis( data, sort_idxs, axis=1)
    data = np.concatenate((data,mask), axis=-1)
    return data

class generator:
    def __init__(self, file, regime, batch_size, w_mu, w_nu, 
                 set_up_Q_lim, up_Q_lim, set_low_Q_lim, low_Q_lim, 
                 apply_add_gauss, g_add_stds, apply_mult_gauss, q_noise_fraction, is_mask = True):
        self.file = file
        self.regime = regime
        self.batch_size = batch_size
        with h5.File(self.file, 'r') as hf:
            self.num = hf[self.regime + '/data/data'].shape[0]
            self.batch_num = self.num // self.batch_size
            self.gen_num = self.batch_num * self.batch_size
            self.w_mu = w_mu
            self.w_nu = w_nu

            #For noise
            self.set_up_Q_lim = set_up_Q_lim
            self.set_low_Q_lim = set_low_Q_lim
            self.is_mask = is_mask
            self.apply_add_gauss = apply_add_gauss
            self.apply_mult_gauss = apply_mult_gauss
            self.g_add_stds = g_add_stds
            if set_up_Q_lim:
                Q_mean = hf['norm_param/mean'][0]
                Q_std = hf['norm_param/std'][0]
                self.Q_up_lim_norm = (up_Q_lim-Q_mean)/Q_std
            if set_low_Q_lim:
                self.means = hf['norm_param/mean'][()]
                self.stds = hf['norm_param/std'][()]
                self.Q_low_lim_norm = (low_Q_lim-self.means[0])/self.stds[0]
                self.aux_idxs = hf['aux_mask/idxs'][()]
                self.aux_vals = hf['aux_mask/vals'][()]
            if apply_mult_gauss:
                Q_mean = hf['norm_param/mean'][0]
                Q_std = hf['norm_param/std'][0]
                self.mult_gauss = gauss_mult_noise(Q_mean/Q_std, q_noise_fraction)
            self.batch_num = self.num // self.batch_size
        
    def step(self, start, stop):
        with h5.File(self.file, 'r') as hf:
            data = hf[self.regime + '/data/data'][start:stop]
            if self.is_mask:
                mask = hf[self.regime + '/mask/data'][start:stop]
                mask = np.expand_dims(mask,axis = -1)
                data = np.concatenate((data,mask),axis = -1)
            labels = np.zeros((self.batch_size, 2))
            ids = hf[self.regime + '/ev_ids/data'][start:stop]  # id of event - starting with 'nu' or 'mu'
            ids = np.array([i[0] for i in ids]).reshape(ids.shape[0],1)
            labels[:] = np.where(ids == 110, [0,1], [1,0])  # 110 - byte code for letter 'n'
            weights = np.where(labels[..., 0] == 1, self.w_mu, self.w_nu)
            if self.set_up_Q_lim:
                data[:,:,0:1] = np.where( data[:,:,0:1]*mask>self.Q_up_lim_norm, self.Q_up_lim_norm, data[:,:,0:1] )
            # apply noise
            if self.apply_add_gauss:
                data = add_gauss(data, self.g_add_stds)
            if self.apply_mult_gauss:
                data[:,:,0] = self.mult_gauss.make_noise(data[:,:,0], data[:,:,-1])
            return (data, labels, weights)
        
    def __call__(self):
        start = 0
        stop = self.batch_size
        for i in range(self.batch_num):
            out_data = self.step(start,stop)
            yield out_data
            start+=self.batch_size
            stop+=self.batch_size

In [3]:
#is mask
is_mask = True

# gauss noise
# 0.1 ~ 1 p.e., 150 ns, 4, 4, 15 m
apply_add_gauss = True
# normal
g_add_stds = [0.03, 0.005, 0.005, 0.005, 0.00003]
# bigger
#stds_gauss = [0.03, 0.005, 0.02, 0.02, 0.0007]
apply_mult_gauss = True
q_noise_fraction = 0.1

# limiting Q vals
set_up_Q_lim = True
up_Q_lim = 100
set_low_Q_lim= False
low_Q_lim = 0.

name = 'baikal_mu-nu_h5-s2_tres08_old_norm.h5'
h5f = '../../../../../ivkhar/Baikal/data/' + name
gen = generator(h5f, 'val', 10, 1, 1, 
                             set_up_Q_lim, up_Q_lim, set_low_Q_lim, low_Q_lim, 
                             apply_add_gauss, g_add_stds, apply_mult_gauss, q_noise_fraction, is_mask)

In [4]:
with h5.File(h5f, 'r') as hf:
    data = hf['val/data/data'][0:20]
data_noise = next(gen())[0]

In [9]:
data_noise[0,:,4]

array([0.7334568 , 0.8396493 , 0.94570625, 1.0519007 , 1.1579926 ,
       3.        , 3.        , 3.        , 3.        , 3.        ,
       3.        , 3.        , 3.        , 3.        , 3.        ,
       3.        , 3.        , 3.        , 3.        , 3.        ,
       3.        , 3.        , 3.        , 3.        , 3.        ,
       3.        , 3.        , 3.        , 3.        , 3.        ,
       3.        , 3.        , 3.        , 3.        , 3.        ,
       3.        , 3.        , 3.        , 3.        , 3.        ,
       3.        , 3.        , 3.        , 3.        , 3.        ,
       3.        , 3.        , 3.        , 3.        , 3.        ,
       3.        , 3.        , 3.        , 3.        , 3.        ,
       3.        , 3.        , 3.        , 3.        , 3.        ,
       3.        , 3.        , 3.        , 3.        , 3.        ,
       3.        , 3.        , 3.        , 3.        , 3.        ,
       3.        , 3.        , 3.        , 3.        , 3.     

In [10]:
data[0,:,4]

array([0.73344314, 0.83958685, 0.94573843, 1.0518894 , 1.1579981 ,
       3.        , 3.        , 3.        , 3.        , 3.        ,
       3.        , 3.        , 3.        , 3.        , 3.        ,
       3.        , 3.        , 3.        , 3.        , 3.        ,
       3.        , 3.        , 3.        , 3.        , 3.        ,
       3.        , 3.        , 3.        , 3.        , 3.        ,
       3.        , 3.        , 3.        , 3.        , 3.        ,
       3.        , 3.        , 3.        , 3.        , 3.        ,
       3.        , 3.        , 3.        , 3.        , 3.        ,
       3.        , 3.        , 3.        , 3.        , 3.        ,
       3.        , 3.        , 3.        , 3.        , 3.        ,
       3.        , 3.        , 3.        , 3.        , 3.        ,
       3.        , 3.        , 3.        , 3.        , 3.        ,
       3.        , 3.        , 3.        , 3.        , 3.        ,
       3.        , 3.        , 3.        , 3.        , 3.     